In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd / content/gdrive/MyDrive/cs394n_project/CS394N
! pip3 install -r requirements.txt

In [ ]:
# Update path for custom module support in Google Colab
import sys
sys.path.append('/content/gdrive/MyDrive/cs394n_project/CS394N/src')

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np

from torch.utils.data import DataLoader

from torchvision import transforms
from torchvision.datasets import CIFAR10, FashionMNIST

#from tqdm.autonotebook import tqdm, trange

from utils.nets import *
from utils.model_tools import train, test, get_recall_per_epoch
from utils.dataset_tools import split_training_data, reorder_classes

KeyboardInterrupt: 

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Data Preparation

In [ ]:
model_selection = 'cnn' # linear | cnn | cnn-demo | vgg
dataset_selection = 'cifar10' # cifar10 | fashionmnist

In [4]:
if dataset_selection == 'fashionmnist':
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))]) # Images are grayscale -> 1 channel
else:
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

## Load Dataset

In [5]:
if dataset_selection == 'cifar10':
    train_data = CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_data = CIFAR10(root='./data', train=False, download=True, transform=transform)
elif dataset_selection == 'fashionmnist':
    train_data = FashionMNIST(root='./data', train=True, download=True, transform=transform)
    test_data = FashionMNIST(root='./data', train=False, download=True, transform=transform)
    
total_classes = len(np.unique(train_data.targets))

## Reorder Classes

In [2]:
# FashionMNIST (torchvision & paper)
# ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

# CIFAR-10
# (torchvision): ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# (paper): ['bird', 'deer', 'dog', 'frog', 'horse', 'airplane','ship', 'truck', 'cat', 'automobile']


In [7]:
# CIFAR10 match torchvision with paper
ordering = {
    0:(5, False),
    1:(9, False),
    2: (0, False),
    3: (8, False),
    4: (1, False),
    5: (2, False),
    6: (3, False),
    7: (4, False),
    8: (6, False),
    9: (7, False),
}

In [8]:
targets, classes = reorder_classes(train_data, ordering)
train_data.targets = targets
train_data.classes = classes

targets, classes = reorder_classes(test_data, ordering)
test_data.targets = targets
test_data.classes = classes

## Create Subsets

In [9]:
holdout_classes = [8, 9]
batch_size = 32

In [10]:
included_data, excluded_data = split_training_data(train_data, holdout_classes) 

train_inc_loader = DataLoader(included_data, batch_size=batch_size, shuffle=True, num_workers=2)
train_exc_loader = DataLoader(included_data, batch_size=batch_size, shuffle=True, num_workers=2)

In [11]:
included_data, excluded_data = split_training_data(test_data, holdout_classes)

test_inc_loader = DataLoader(included_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_exc_loader = DataLoader(included_data, batch_size=batch_size, shuffle=True, num_workers=2)

# Train Model

## Load Architecture

In [12]:
num_classes = total_classes - len(holdout_classes)

if model_selection == 'linear':
    input_size = train_data[0][0].shape[0] * train_data[0][0].shape[1] * train_data[0][0].shape[2]
    model = LinearFashionMNIST_alt(input_size, num_classes)
elif model_selection == 'cnn':
    model = CNN_3B(num_classes)
elif model_selction == 'cnn-demo':
    model = CNN_demo(num_classes)
elif model_slection == 'vgg':
    print('Model not implemented')
    
model.to(device)

LinearFashionMNIST_alt(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (input_layer): Linear(in_features=784, out_features=128, bias=True)
  (output_layer): Linear(in_features=128, out_features=8, bias=True)
)

## File Paths

In [13]:
weight_dir = './models/'
log_dir = './logs/'

model_file = weight_dir + model_selection + '_' + dataset_selection + '_' + 'holdout' + '_' + str(holdout_classes) + '.pt'
recall_file = log_dir + model_selection + '_' + dataset_selection + '_' + 'holdout' + '_' + str(holdout_classes) + 'recall.npy'
train_losses_file = log_dir + model_selection + '_' + dataset_selection + '_' + 'holdout' + '_' + str(holdout_classes) + 'train_loss.txt'
test_losses_file = log_dir + model_selection + '_' + dataset_selection + '_' + 'holdout' + '_' + str(holdout_classes) + 'test_loss.txt'

## Hyperparameters

In [14]:
num_epochs = 15

initial_learning_rate = 0.001
final_learning_rate = 0.0001

decay_rate = (final_learning_rate/initial_learning_rate)**(1/num_epochs)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=initial_learning_rate)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decay_rate)

## Training Loop

In [15]:
train_losses = []
test_losses = []
t = range(num_epochs)

for epoch in t:
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loss = train(train_inc_loader, model, loss_fn, optimizer, device)
    test_loss = test(test_inc_loader, model, loss_fn, device)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    
    lr_scheduler.step()
    
torch.save(model.state_dict(), model_file)

#with open(train_losses_file, 'w') as fp:
#    for s in train_losses:
#        fp.write("%s\n" % s)
        
#with open(test_losses_file, 'w') as fp:
#    for x in test_losses:
#        fp.write("%s\n" % x)

print("Done!")

Epoch 1
-------------------------------
loss: 2.149078  [    0/48000]
loss: 0.478415  [32000/48000]
Test Error: 
 Accuracy: 82.2%, Avg loss: 0.476515 

Epoch 2
-------------------------------
loss: 0.658005  [    0/48000]
loss: 0.447382  [32000/48000]
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.465710 

Epoch 3
-------------------------------
loss: 0.571728  [    0/48000]
loss: 0.682355  [32000/48000]
Test Error: 
 Accuracy: 83.0%, Avg loss: 0.464162 

Epoch 4
-------------------------------
loss: 0.248085  [    0/48000]
loss: 0.272078  [32000/48000]
Test Error: 
 Accuracy: 83.3%, Avg loss: 0.446991 

Epoch 5
-------------------------------
loss: 0.281498  [    0/48000]
loss: 0.599176  [32000/48000]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.451948 

Epoch 6
-------------------------------
loss: 0.340229  [    0/48000]
loss: 0.675200  [32000/48000]
Test Error: 
 Accuracy: 82.8%, Avg loss: 0.471944 

Epoch 7
-------------------------------
loss: 0.555285  [    0/48000]
loss: 0.382638  [3

TypeError: not all arguments converted during string formatting

In [ ]:
recalls_loaded = np.load(recall_file)
print(recalls == recalls_loaded)
# plots